In [23]:
COLLECTION_NAME = "web-places"

In [24]:
import os
import sys
sys.path.append("..")
import json
from datetime import datetime
from groq import Groq
from dotenv import load_dotenv
load_dotenv(override=True)
groq = Groq(
    api_key=os.environ.get("GROQ_API_KEY"),
)
from langchain.memory import ConversationBufferMemory
from langchain.chains import ConversationalRetrievalChain
from langchain.prompts import (
    ChatPromptTemplate,
    HumanMessagePromptTemplate,
    SystemMessagePromptTemplate,
)

from langchain_groq import ChatGroq
from utils import database_managers, embedding


llm = ChatGroq(temperature=0, model_name="Llama3-8b-8192")
embedding = embedding.EmbeddingFunction('fast-bgeEmbedding').embedder

Fetching 5 files: 100%|██████████| 5/5 [00:00<?, ?it/s]


In [25]:
vectore_store=qdrantClient = database_managers.QDrantDBManager(
    url=os.getenv('QDRANT_URL'),
    port=6333,
    collection_name=COLLECTION_NAME,
    vector_size=768, #??
    embedding=embedding,
    record_manager_url=r"sqlite:///record_manager_cache.sql"
)
retriever = vectore_store_client=vectore_store.vector_store.as_retriever()

Collection web-places already exists!


In [26]:
system_message_prompt = SystemMessagePromptTemplate.from_template(
    """
        Sei un assistente nella ricerca di ristoranti o locali o eventi. Date le informazioni di contesto restituisci una lista di ristoranti, locali, eventi richiesti,
        che soddisfano i requisiti di ricerca.

        Per ogni elemento indica sempre:
          + nome del ristorante, locale, evento
          + indirizzo
          + descrizione

        Se non riesci a trovare le suddette informazioni scrivi un <informazione non disponibile> al su posto.

        Non rispondere a domande non legate a ristoranti, locali o eventi. 

        Data una domanda, rispondi con la documentazione più rilevante dal contesto di seguito:
        {context}
        """
)
human_message_prompt = HumanMessagePromptTemplate.from_template("{question}")

In [27]:
memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)

In [34]:
memory

ConversationBufferMemory(chat_memory=InMemoryChatMessageHistory(messages=[HumanMessage(content='Dove posso mangiare in montagna?'), AIMessage(content='Ecco alcuni ristoranti in montagna che ti consiglio:\n\n1. **KRO** - Via Tollarini 70/C, Temù (BS) - Un ristorante in stile montano con cucina basata su ingredienti locali e proposte di carne e pesce.\n2. **RIFUGIO IL PASTORE** - Un ristorante con cucina tradizionale e vista panoramica sulla montagna.\n\nSpero che questo ti sia stato utile!'), HumanMessage(content="Qual'è l'ultimo locale che mi hai consigliato?"), AIMessage(content="I'm happy to help! According to the text, the last restaurant I recommended is Trattoria San Filippo Neri. Here's the information:\n\n* Name: Trattoria San Filippo Neri\n* Address: Viale Monza 220, Milano\n* Description: A historic trattoria with simple and genuine dishes and a friendly and cozy atmosphere.")]), return_messages=True, memory_key='chat_history')

In [28]:
conversation_chain = ConversationalRetrievalChain.from_llm(
        llm=llm,
        retriever=retriever,
        memory=memory,
        combine_docs_chain_kwargs={
            "prompt": ChatPromptTemplate.from_messages(
                [
                    system_message_prompt,
                    human_message_prompt,
                ]
            ),
        },
    )

In [29]:
res = conversation_chain.invoke({"question": "Dove posso mangiare in montagna?"})

In [31]:
print(res['answer'])

Ecco alcuni ristoranti in montagna che ti consiglio:

1. **KRO** - Via Tollarini 70/C, Temù (BS) - Un ristorante in stile montano con cucina basata su ingredienti locali e proposte di carne e pesce.
2. **RIFUGIO IL PASTORE** - Un ristorante con cucina tradizionale e vista panoramica sulla montagna.

Spero che questo ti sia stato utile!


In [32]:
res = conversation_chain.invoke({"question": "Qual'è l'ultimo locale che mi hai consigliato?"})

In [33]:
print(res['answer'])

I'm happy to help! According to the text, the last restaurant I recommended is Trattoria San Filippo Neri. Here's the information:

* Name: Trattoria San Filippo Neri
* Address: Viale Monza 220, Milano
* Description: A historic trattoria with simple and genuine dishes and a friendly and cozy atmosphere.
